# 誤差関数の区間演算
誤差関数
$$
\mathrm{erf}(x) = \frac{2}{\sqrt{\pi}}\int_0^x e^{-t^2} dt
$$
の被積分関数のChebyshev補間(区間演算)とその補間誤差の計算を行う。

In [1]:
versioninfo()

Julia Version 1.10.0
Commit 3120989f39b (2023-12-25 18:01 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  CPU: 8 × Apple M2
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, apple-m1)
  Threads: 5 on 4 virtual cores
Environment:
  JULIA_NUM_THREADS = 4


In [2]:
include("IntervalCheb.jl")

chebmin (generic function with 2 methods)

In [3]:
f(x) = exp(-x^2) #erf関数の被積分関数: 2 / sqrt(π)は後でかける
f(x::Interval{T}) where {T<:Real} = exp(-x^interval(2)) #*(interval(2) / sqrt(interval(π)))
f(x::Complex{Interval{T}}) where {T<:Real} = exp(-x * x)

f (generic function with 3 methods)

## 不定積分による実装 `erf1(x)`
手順

1. 誤差関数の被積分関数 $f(t) = \frac{2}{\sqrt{\pi}}e^{-t^2}$ に対して $[0,L]$ ($x<L$, $x$は誤差関数に対する入力値)で区間チェビシェフ補間 $p(t)$ を構成する。
1. 区間チェビシェフ補間の不定積分 $P(t)=\int p(t)dt$ を $P(0)=0$ となるよう区間演算で計算する。
1. 不定積分 $P(t)$ に $t=x$ における区間チェビシェフ補間の関数値を区間演算する。


Step 1. 区間チェビシェフ補間を構成する。

In [4]:
dom = [0, 6] # L = 6
ia = interval_cheb(f, dom)
radius.(ia)
# include("IntervalCheb.jl")
# setprecision(BigFloat, 102)
# ia = biginterval_cheb(f, dom, ϵ=interval(eps(BigFloat)),tolerance=5e-23)
# radius.(ia)

40-element Vector{Float64}:
 9.120482147295661e-14
 4.440892098500626e-16
 3.3306690738754696e-16
 3.0531133177191805e-16
 1.3183898417423734e-16
 2.498001805406602e-16
 1.942890293094024e-16
 2.8102520310824275e-16
 1.214306433183765e-16
 2.8015784137025435e-16
 ⋮
 2.8896601897334286e-16
 9.020562075079397e-17
 2.889660189668805e-16
 2.2317900473716653e-16
 2.5163781088123246e-16
 1.275777258187477e-16
 2.385147739033042e-16
 1.896699485441491e-16
 2.7416228160769545e-16

Step 2. 不定積分を計算する

In [5]:
iA = chebindefint(ia, dom)
# radius.(iA)
# plot_cheb(mid.(iA),I=dom)
# plot!(erf,dom[1],dom[2])

41-element Vector{Interval{Float64}}:
 [0.723427, 0.723428]_com
 [0.297923, 0.297924]_com
 [-0.226424, -0.226423]_com
 [0.138759, 0.13876]_com
 [-0.0631519, -0.0631518]_com
 [0.0151949, 0.015195]_com
 [0.00530273, 0.00530274]_com
 [-0.00825034, -0.00825033]_com
 [0.00460028, 0.00460029]_com
 [-0.00104253, -0.00104252]_com
   ⋮
 [3.17042e-12, 3.17048e-12]_com
 [-5.19598e-13, -5.19568e-13]_com
 [-2.35906e-13, -2.35857e-13]_com
 [1.21814e-13, 1.21845e-13]_com
 [-4.44515e-15, -4.40429e-15]_com
 [-1.23571e-14, -1.23313e-14]_com
 [3.44841e-15, 3.48889e-15]_com
 [5.21975e-16, 5.36566e-16]_com
 [-4.44398e-16, -4.23835e-16]_com

Step 3. $t=x$ における関数値を区間演算する。

In [6]:
# erf1(x) = eval_interval_cheb(iA, x, I=dom) * (interval(2) / sqrt(interval(π)))
erf1(x::T) where {T} = eval_interval_cheb(iA, x, dom) * (interval(2) / sqrt(interval(T,π)))

erf1 (generic function with 1 method)

In [7]:
using SpecialFunctions
x = (.1)
@show erf(x)
@show (erf1(x));

erf(x) = 0.1124629160182849
erf1(x) = Interval{Float64}(0.11246291601766975, 0.11246291601890017, com)


In [8]:
xx = dom[1]:0.1:dom[2]
issubset_interval(interval(erf.(xx)), erf1.(xx))

true

In [9]:
# maximum(radius, erf1.((xx)))
maximum(radius, ia)

9.120482147295661e-14

以上をまとめて `erf1` という関数を作成する。

In [10]:
dom = [0.0, 6.0]
ia = interval_cheb(f, dom, ϵ=interval(2^-67))
iA = chebindefint(ia, dom)
function erf_point(x::Float64, dom, iA)
    if x > 5.864
        return interval(1-2^-53,1)
    elseif x < -5.864
        return interval(-1, -1+2^-53)
    # elseif dom[2] < x
    #     # dom = [0, x]
    #     # iA = chebindefint(interval_cheb(f, dom), I=dom)
    #     # return eval_interval_cheb(iA, x, I=dom) * (interval(2) / sqrt(interval(π)))
    #     dom = [0, x]
    #     ia = interval_cheb(f, dom)
    #     return chebint(ia, dom) * (interval(2) / sqrt(interval(π)))
    # elseif x < -dom[2]
    #     # dom = [0, -x]
    #     # iA = chebindefint(interval_cheb(f, dom), I=dom)
    #     # return -eval_interval_cheb(iA, -x, I=dom) * (interval(2) / sqrt(interval(π)))
    #     dom = [0, -x]
    #     ia = interval_cheb(f, dom)
    #     return chebint(ia, dom) * (interval(2) / sqrt(interval(π)))
    else
        if x == 0
            return 0
        elseif x > 0
            return eval_interval_cheb(iA, x, dom) * (interval(2) / sqrt(interval(π)))
        else
            return -eval_interval_cheb(iA, -x, dom) * (interval(2) / sqrt(interval(π)))
        end
    end
end
erf1(x) = erf_point(x, dom, iA)

erf1 (generic function with 1 method)

## 定積分による実装 `erf2(x)`
手順

1. 誤差関数の被積分関数 $f(t) = \frac{2}{\sqrt{\pi}}e^{-t^2}$ を $[0,x]$ ($x$ は誤差関数に対する入力値)において区間チェビシェフ補間 $p(t)$ を構成する。
1. 区間チェビシェフ補間 $p(t)$ の定積分 $\int_0^x f(t)dt$ を区間演算に基づき計算する。


In [11]:
function erf2(x)
    if x == 0
        return 0
    elseif x > 0
        dom = [0, x]
        ia = interval_cheb(f, dom, ϵ=interval(2^-67)) # Step 1
        return chebint(ia, dom) * (interval(2) / sqrt(interval(π))) # Step 2
    else
        dom = [0, -x]
        ia = interval_cheb(f, dom, ϵ=interval(2^-67))
        return -chebint(ia, dom) * (interval(2) / sqrt(interval(π)))
    end
end

erf2 (generic function with 1 method)

上の方法と同じであるが、変数変換をして
$$
    \mathrm{erf}(x) = \frac{2}{\sqrt{\pi}}\int_0^x {e}^{-t^2}{d}t =\frac{x}{\sqrt{\pi}}\int_{-1}^1 {e}^{-(x(\xi+1)/2)^2}{d}\xi
$$
という形で計算する方法もある。これを `erf3(x)` として実装する。

In [12]:
function erf3(x)
    g(ξ) = f(x * (ξ + 1) / 2)
    g(ξ::Interval{T}) where {T<:Real} = f(interval(x) * (ξ + interval(1)) / interval(2))
    g(ξ::Complex{Interval{T}}) where {T<:Real} = f(interval(x) * (ξ + interval(1)) / interval(2))
    ia = interval_cheb(g, ϵ=interval(2^-67))
    return chebint(ia) * interval(x) / sqrt(interval(π))
end

erf3 (generic function with 1 method)

### 各方法 `erf1`, `erf2`, `erf3` を比較してみる

In [13]:
x = -2.23
@show erf(x)
@show (erf1(x))
@show (erf2(x))
@show (erf3(x));

erf(x) = -0.9983878320616982
erf1(x) = Interval{Float64}(-0.998387832061743, -0.9983878320616529, com)
erf2(x) = Interval{Float64}(-0.9983878320617238, -0.9983878320616729, com)
erf3(x) = Interval{Float64}(-0.9983878320617237, -0.998387832061673, com)


区間$[-6,6]$上のランダムに与えた150点に対して、誤差関数の関数値を区間演算で計算し、その区間半径の最大と実行時間を計算する。

In [14]:
# xx = -6:0.1:6
n = 150
xx = 6(2rand(n).-1)
@time @show maximum(radius, erf1.(xx))
@time @show maximum(radius, erf1.(xx))
@time @show maximum(radius, erf2.(xx))
@time @show maximum(radius, erf2.(xx))
@time @show maximum(radius, erf3.(xx))
@time @show maximum(radius, erf3.(xx));

maximum(radius, erf1.(xx)) = 6.52811138479592e-14
  0.022278 seconds (63.67 k allocations: 4.672 MiB, 91.04% compilation time)
maximum(radius, erf1.(xx)) = 6.52811138479592e-14
  0.001969 seconds (1.51 k allocations: 420.523 KiB)
maximum(radius, erf2.(xx)) = 1.8073320617872923e-12
  0.092604 seconds (887.72 k allocations: 60.403 MiB, 4.77% gc time, 52.83% compilation time)
maximum(radius, erf2.(xx)) = 1.8073320617872923e-12
  0.042867 seconds (767.66 k allocations: 51.903 MiB, 8.83% gc time)
maximum(radius, erf3.(xx)) = 1.8073320617872923e-12
  0.055268 seconds (821.03 k allocations: 54.856 MiB, 6.44% gc time, 21.27% compilation time)
maximum(radius, erf3.(xx)) = 1.8073320617872923e-12
  0.043786 seconds (767.34 k allocations: 51.169 MiB, 9.35% gc time)


計算時間は事前に計算しておいた不定積分のチェビシェフ係数を利用できる `erf1` が最も早い。計算制度についても `erf1` が最も高精度であることが確認された。

以上から `IntervalArithmetic.jl` を使った `erf` 関数の区間演算の実装は以下のようになる。誤差関数は単調増加関数のため、入力の区間の端点で誤差関数の値を区間演算して、それぞれ下端、上端にすることで区間演算は実装できる。

In [15]:
dom = [0, 6]
ia = interval_cheb(f, dom)
iA = chebindefint(ia, dom)
function erf_point(x::Float64, dom, iA)
    if x > 5.864
        return interval(1-2^-53,1)
    elseif x < -5.864
        return interval(-1, -1+2^-53)
    # elseif dom[2] < x
    #     # dom = [0, x]
    #     # iA = chebindefint(interval_cheb(f, dom), I=dom)
    #     # return eval_interval_cheb(iA, x, I=dom) * (interval(2) / sqrt(interval(π)))
    #     dom = [0, x]
    #     ia = interval_cheb(f, dom)
    #     return chebint(ia, dom) * (interval(2) / sqrt(interval(π)))
    # elseif x < -dom[2]
    #     # dom = [0, -x]
    #     # iA = chebindefint(interval_cheb(f, dom), I=dom)
    #     # return -eval_interval_cheb(iA, -x, I=dom) * (interval(2) / sqrt(interval(π)))
    #     dom = [0, -x]
    #     ia = interval_cheb(f, dom)
    #     return chebint(ia, dom) * (interval(2) / sqrt(interval(π)))
    else
        if x == 0
            return 0
        elseif x > 0
            return eval_interval_cheb(iA, x, dom) * (interval(2) / sqrt(interval(π)))
        else
            return -eval_interval_cheb(iA, -x, dom) * (interval(2) / sqrt(interval(π)))
        end
    end
end

import SpecialFunctions: erf
erf(x::Interval{T}) where {T<:Real} = interval(erf_point(inf(x), dom, iA), erf_point(sup(x), dom, iA))
ierf(x) = erf_point(x, dom, iA)

ierf (generic function with 2 methods)

In [16]:
erf(-1), erf(2)

(-0.8427007929497149, 0.9953222650189527)

In [17]:
erf(interval(-1, 2))

[-0.842701, 0.995323]_com

In [18]:
@time radius(ierf(2.))

  0.002018 seconds (170 allocations: 9.875 KiB, 98.28% compilation time)


4.214406601477094e-13

## 誤差関数の区間演算（引数が複素数の時）

In [19]:
include("IntervalCheb.jl")

chebmin (generic function with 2 methods)

$$
    \mathrm{erf}(z) = \frac{2}{\sqrt{\pi}}\int_0^z {e}^{-t^2}{d}t =\frac{z}{\sqrt{\pi}}\int_{-1}^1 {e}^{-(z(\xi+1)/2)^2}{d}\xi
$$

In [20]:
function interval_chebcoeffs_complex(f, M, I=[-1, 1])
    a = I[1]
    b = I[2]
    n = M - 1
    cpts = interval_chebpts(n, a, b)
    fvals = f.(cpts)
    FourierCoeffs = verifyfft([reverse(fvals); fvals[2:end-1]]) # the length of this must be power of 2
    ChebCoeffs = FourierCoeffs[1:n+1] / interval(n)
    ChebCoeffs[1] = ChebCoeffs[1] / interval(2)
    ChebCoeffs[end] = ChebCoeffs[end] / interval(2)
    return ChebCoeffs # return Two-sided Chebyshev
end

interval_chebcoeffs_complex (generic function with 2 methods)

In [21]:
function interval_cheb_complex(f, I=[-1, 1]; ϵ=interval(2^-52), div=2^-3) # for general func
    a = cheb_complex(f, I)
    M = length(a) # Set M
    M̃ = nextpow(2, M) + 1 # Set M̃
    ia = interval_chebcoeffs_complex(f, M̃, I) # Coeffs of p̃(x)
    # Truncation error is in the zero mode
    ia = truncCheb(ia, M) # Coeffs of Πₘp̃(x)
    # Set rho of Bernstein ellipse
    rho = ϵ^(-interval(1) / (interval(M̃) - interval(1)))
    I1 = interval(I[1])
    I2 = interval(I[2])
    i1 = interval(1)
    i2 = interval(2)
    g(ξ) = f((i1 - ξ) / i2 * I1 + (i1 + ξ) / i2 * I2)
    fz = fzeval(g, rho, div) # Evaluate f(z)
    # Interpolation error via Bernstein ellipse is also in the zero mode
    err = (interval(4) * rho^(-(interval(M̃) - interval(1))) / (rho - interval(1))) * interval(fz)
    # midrad form of interval Cheb interpolation
    ia[1] = ia[1] + interval(0, err; format=:midpoint)
    return ia
end

interval_cheb_complex (generic function with 2 methods)

In [22]:
function chebint(a::Vector{Complex{Interval{T}}}; I=[-1, 1]) where {T<:Real} # Input is Two-sided
    M = length(a)
    n = interval(Vector(0:2:M-1))
    # @show sum(2*a[1:2:end]./(1.0 .- n.^2))*((I[2]-I[1])/2)
    i2 = interval(2.0)
    i1 = interval(1.0)
    return sum(i2 * a[1:2:end] ./ (i1 .- n .^ i2)) * ((interval(I[2]) - interval(I[1])) / i2)
end

chebint (generic function with 5 methods)

In [23]:
iz = interval(1.0 + 0.1im)
z = mid(iz)


1.0 + 0.1im

In [24]:
f(x) = exp(-x^2) #erf関数の被積分関数: 2 / sqrt(π)は後でかける
f(x::Interval{T}) where {T<:Real} = exp(-x^interval(2)) #*(interval(2) / sqrt(interval(π)))
f(x::Complex{Interval{T}}) where {T<:Real} = exp(-x * x)
g(ξ) = f((z / 2) .* (ξ + 1))
g(ξ::Interval{T}) where {T<:Real} = f((iz / interval(2)) .* (ξ + interval(1)))
g(ξ::Complex{Interval{T}}) where {T<:Real} = f((iz / interval(2)) .* (ξ + interval(1)))


g (generic function with 3 methods)

In [25]:
ia = interval_cheb_complex(g)
# chebint(ia)
(interval(z) / sqrt(interval(π))) * chebint(ia)

[0.846858, 0.846859]_com + ([0.0413716, 0.0413717]_com)im

In [26]:
erf(z)

0.8468587817220126 + 0.04137168721306572im

In [27]:
function erf(iz::Complex{Interval{T}}) where {T<:Real}
    z = mid(iz)
    f(x) = exp(-x^2) #erf関数の被積分関数: 2 / sqrt(π)は後でかける
    f(x::Interval{T}) where {T<:Real} = exp(-x^interval(2)) #*(interval(2) / sqrt(interval(π)))
    f(x::Complex{Interval{T}}) where {T<:Real} = exp(-x * x)
    g(ξ) = f((z / 2) .* (ξ + 1))
    g(ξ::Interval{T}) where {T<:Real} = f((iz / interval(2)) .* (ξ + interval(1)))
    g(ξ::Complex{Interval{T}}) where {T<:Real} = f((iz / interval(2)) .* (ξ + interval(1)))
    return (interval(z) / sqrt(interval(π))) * chebint(interval_cheb_complex(g))
end

ierf(z::Complex{T}) where {T<:Real} = erf(interval(z))

ierf (generic function with 2 methods)

In [28]:
r = randn(5)
i = randn(5)
z = Complex.(r, i)
iz = interval(z)
erf.(iz)

5-element Vector{Complex{Interval{Float64}}}:
                                            [-1.06766, -1.06765]_com + ([0.0221192, 0.0221193]_com)im
                                              [0.651669, 0.65167]_com + ([0.213857, 0.213858]_com)im
 [-0.136954, -0.136953]_com + ([-0.723649, -0.723648]_com)im
  [-0.362496, -0.362495]_com + ([-1.00451, -1.0045]_com)im
                                               [1.20031, 1.20032]_com + ([0.062666, 0.0626661]_com)im

In [29]:
erf.(z)

5-element Vector{ComplexF64}:
  -1.0676534401094917 + 0.0221192204696726im
   0.6516697119747095 + 0.21385781957176478im
 -0.13695347241795452 - 0.7236481766523777im
  -0.3624957847444986 - 1.00450427789555im
   1.2003121283880411 + 0.06266604601336613im